## A/B Testing 的常見錯誤 - early stopping
- 一般來說，A/B Testing會在設定好`型一錯誤alpha`、`power(1-beta)`以及`Minimum Detectable Effect`的狀況下，設定預估收集的樣本數，在依此收集足夠的樣本`後`透過檢定判斷是否顯著。
- 而`early stopping`指的是，當你在收集完足夠樣本數之前，提前`peek`目前的測試效果，提早做檢定導致`偽陽性`發生，也就是實際沒有顯著差異，卻因為沒有收集到足夠的樣本，導致誤判。

---

[內容參考 - A/B Testing：「偷看結果」將成為最大的錯誤](https://haosquare.com/ab-testing-peeking/)

In [24]:
import numpy as np
import pandas as pd
from scipy import stats

In [25]:
## 次日留存率範例，今天首日安裝，隔天會再使用的比例

# 列聯表、卡方檢定用
contingency_table = np.array([
    [8000, 20000-8000], # 代表次數統計
    [8200, 20000-8200]  # 同上
])

df = pd.DataFrame(contingency_table, index=['A組(控制組)', 'B組(實驗組)'], columns=['次日留存', '非次日留存'])
df

,次日留存,非次日留存
A組(控制組),8000,12000
B組(實驗組),8200,11800


In [26]:
## 卡方檢定是否B組比A組好

print(f"卡方簡定: p-value: {stats.chi2_contingency(contingency_table)[1]}")

卡方簡定: p-value: 0.04267036646823683


P-value < 0.05，結論 B 組確實較好、採用 B 組為 APP 新設計

### 錯誤實驗導致錯誤決策：A/A Testing with early stopping
- A/A Testing: 在A/B testing之前先透過隨機分配的統計檢定確認`隨機真的是隨機(無顯著差異)`，以免隨機抽樣本身就有偏誤。

In [27]:
## 累加

np.cumsum([10000]*4)

array([10000, 20000, 30000, 40000], dtype=int32)

In [12]:
## 不同樣本數的bernuli分配: 代表留存(搭配上面的例子)

np.cumsum(np.random.binomial(n=10000, p=0.4, size=4))

array([ 3974,  7994, 12030, 16043], dtype=int32)

In [13]:
## 同上

np.cumsum(np.random.binomial(n=10000, p=0.4, size=4))

array([ 3999,  7998, 11910, 15859], dtype=int32)

In [18]:
## 演示 early stopping 透過模擬的方式。
## 以下a, b可以知道 p=0.4 其實本質上是沒有差異的！

np.random.seed(45)
n_trials = np.cumsum([10000]*4)
a_trials = np.cumsum(np.random.binomial(n=10000, p=0.4, size=4))
b_trials = np.cumsum(np.random.binomial(n=10000, p=0.4, size=4))

## 不同樣本大小的列聯表 -> 卡方檢定
test_for_trials = [
  stats.chi2_contingency(np.array([[a_trials[i], n_trials[i]-a_trials[i]], [b_trials[i], n_trials[i]-b_trials[i]]]))[1] # 取得pvalue
    for i in range(4)]
print("A/B 兩組實際上並無 KPI 成效差異，然而樣本數不足就提早結束實驗，更容易受 P-Value 誤導，導致錯誤決策")
df = pd.DataFrame({"總共樣本數": n_trials, 
          "A 組別的KPI": a_trials / n_trials, 
          "B 組別的KPI": b_trials / n_trials,
          "P-Value": test_for_trials})

df

A/B 兩組實際上並無 KPI 成效差異，然而樣本數不足就提早結束實驗，更容易受 P-Value 誤導，導致錯誤決策


,總共樣本數,A 組別的KPI,B 組別的KPI,P-Value
0,10000,0.41180,0.395100,0.016728
1,20000,0.40745,0.403650,0.445016
2,30000,0.40540,0.404333,0.796543
3,40000,0.40380,0.404025,0.954033


> 可以發現隨著樣本數增加，P-Value不斷增加，就比較不會有誤判的可能，因此越提早查看越容易發生錯誤。

### 那麼來看看越早偷看造成的影響為何！
- 透過每累積5%的樣本就偷看一次，若顯著就結束實驗，去查看`false positive`機率。
- 透過每累積10%的樣本就偷看一次，若顯著就結束實驗，去查看`false positive`機率。

In [28]:
## 每5%就偷看
## DRY

needed_sample_size = 40000
np.random.binomial(n=10000, p=0.4, size=1)

array([4060])

## 結論
- 不要偷看，等到樣本數累積到足夠才做判斷！